<a href="https://colab.research.google.com/github/Ubaid-Manzoor/CatVsDog_ImageClassifier/blob/master/DogVsCat_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#This Block Of Code Only Needed in Google Colab

# from google.colab import files
# files.upload()

In [0]:
#This Block Of Code Only Needed in Google Colab

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle

# #To Avoid Warning
# !chmod 600 ~/.kaggle/kaggle.json

In [0]:
#This Block Of Code Only Needed in Google Colab

# !kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../content"))

# Any results you write to the current directory are saved as output.

['.config', 'train.zip', 'test.zip', 'sample_submission.csv', 'kaggle.json', 'sample_data']


In [0]:
#This Block Of Code Only Needed in Google Colab

!unzip "/content/train.zip"

In [0]:
#This Block Of Code Only Needed in Google Colab

!unzip "/content/test.zip"

In [0]:
import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Activation , Flatten , Conv2D , MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [0]:
#Give Model a Name

name = "CatVsDog-{}".format(int(time.time()))

In [0]:
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(name))

In [0]:
traindir = "train"
testdir = "test"
def get_ytrain(traindir):
    y_train = []
    for img_name in os.listdir(traindir):
        animal_name = []
        animal_name = img_name.split('.')[0]
        y_train.append(0 if animal_name == 'cat' else 1)
    return y_train
def get_data(dir_path="train" , img_size=100 , data_type="train"):
    data = []
    if data_type == "train":
        y_train = get_ytrain(dir_path)
    for index,img in enumerate(os.listdir(dir_path)):
        img_array = cv2.imread(os.path.join(dir_path,img))
        new_array = cv2.resize(img_array , (img_size , img_size))
        if data_type=="train":
            data.append([new_array , y_train[index]])
        else:
            data.append(new_array)
    return data

In [0]:
training_data = get_data(img_size=100)

In [41]:
test_data = get_data(dir_path = testdir , img_size=100,data_type="test")
test_data = np.array(test_data)
print(test_data.shape)

(12500, 100, 100, 3)


In [0]:
X_train = []
y_train = []

In [0]:
for feature , label in training_data:
    X_train.append(feature)
    y_train.append(label)

In [44]:
X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1,1)
print(X_train.shape)
print(y_train.shape)

(25000, 100, 100, 3)
(25000, 1)


In [0]:
# #Save Data
# def save_data(file_name , data_to_dump):
#     with open(file_name , mode='wb') as data:
#         pickle.dump(data_to_dump , data)

# save_data("X_train" , X_train)
# save_data("y_train" , y_train)
# save_data("test_data" , test_data)

In [0]:
# #Retreive Data Back 
# def load_data(file_name):
#     with open(file_name,mode='rb') as file:
#         data = pickle.load(file)
#     return data
# X_train = load_data("X_train")
# print(X_train.shape)
# y_train = load_data("y_train")
# print(y_train.shape)
# test_data = load_data("test_data")
# print(test_data.shape)

In [0]:
X_train = X_train/255.0
test_data = test_data/255.0

In [66]:
#model
model = Sequential()
model.add((Conv2D(filters=32 , kernel_size=(8,8) , input_shape = X_train.shape[1:])))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Dropout(0.3))

model.add((Conv2D(filters=64 , kernel_size=(6,6))))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

# model.add((Conv2D(filters=128 , kernel_size=(4,4))))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

model.add(Flatten())
# model.add(Dense(16))
# model.add(Activation("relu"))
# model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy',
             optimizer="adam",
             metrics=['accuracy'])
model.fit(X_train , y_train ,epochs=5
          ,  batch_size=64,validation_split=0.2 , callbacks=[tensorboard])

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 13s 658us/step - loss: 0.6466 - acc: 0.6237 - val_loss: 0.5762 - val_acc: 0.7080
Epoch 2/5
20000/20000 [==============================] - 13s 637us/step - loss: 0.5670 - acc: 0.7107 - val_loss: 0.5292 - val_acc: 0.7468
Epoch 3/5
20000/20000 [==============================] - 13s 641us/step - loss: 0.5264 - acc: 0.7410 - val_loss: 0.5066 - val_acc: 0.7574
Epoch 4/5
20000/20000 [==============================] - 13s 640us/step - loss: 0.4984 - acc: 0.7583 - val_loss: 0.5129 - val_acc: 0.7426
Epoch 5/5
20000/20000 [==============================] - 13s 644us/step - loss: 0.4779 - acc: 0.7735 - val_loss: 0.5281 - val_acc: 0.7232


In [0]:
# prediction = model.predict_classes(test_data)

In [53]:
# prediction.shape

(12500, 1)

In [0]:
# prediction = prediction.reshape(prediction.shape[0],)

In [55]:
# counter = range(1, len(test_data) + 1)
# solution = pd.DataFrame({"id": counter, "label":list(prediction)})
 

# solution.to_csv("dogsVScats.csv", index = False)
# solution

,id,label
0,1,0
1,2,0
2,3,1
3,4,0
4,5,0
5,6,0
6,7,0
7,8,1
8,9,1
9,10,1
